In [1]:
import pandas as pd
import numpy as np

In [2]:
num_players = 5
deck_df = pd.read_csv('deck.csv')
hand_rankings_df = pd.read_csv('poker_hand_rankings.csv')
card_rankings_df = pd.read_csv('card_rankings.csv')
# Initialize empty dictionary to store player hands
player_hands = {i: [] for i in range(num_players)}
# Initialize empty dictionary to store table cards (5 slots)
table_cards = {i: [] for i in range(5)}



In [3]:
table_cards

{0: [], 1: [], 2: [], 3: [], 4: []}

In [4]:
player_hands

{0: [], 1: [], 2: [], 3: [], 4: []}

In [5]:
def deck_shuffler(deck_df):
    shuffled_df = deck_df.copy()
    shuffled_df = shuffled_df.iloc[np.random.permutation(len(shuffled_df))]
    return shuffled_df

def initial_deal(shuffled_df, player_hands):
    # Deal 2 cards to each player
    for card_num in range(2):  # Deal 2 rounds
        for player in player_hands.keys():  # Deal to each player
            # Get the next card (first row of shuffled deck)
            card = shuffled_df.iloc[player + (card_num * len(player_hands))]
            # Add card to player's hand as [suit, rank] list
            player_hands[player].append([card['suit'], card['rank']])
    
    # Remove dealt cards from the deck (2 cards * number of players)
    remaining_deck = shuffled_df.iloc[2 * len(player_hands):].reset_index(drop=True)
    
    return player_hands, remaining_deck


def burn_card(remaining_deck):
    # Burn one card from the remaining deck
    burned_card = remaining_deck.iloc[0]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return remaining_deck

def deal_flop(remaining_deck, table_cards):
    # Deal 3 cards to the table
    for i in range(3):
        card = remaining_deck.iloc[0]
        # Add card to table as [suit, rank] list, similar to player hands
        table_cards[i] = [card['suit'], card['rank']]
        remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def deal_turn(remaining_deck, table_cards):
    # Deal 1 card to the table in position 3
    card = remaining_deck.iloc[0]
    # Add card to table as [suit, rank] list
    table_cards[3] = [card['suit'], card['rank']]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def deal_river(remaining_deck, table_cards):
    # Deal 1 card to the table in position 4
    card = remaining_deck.iloc[0]
    # Add card to table as [suit, rank] list
    table_cards[4] = [card['suit'], card['rank']]
    remaining_deck = remaining_deck.iloc[1:].reset_index(drop=True)
    return table_cards, remaining_deck

def assemble_hand(player_hands, table_cards):
    # Dictionary to store combined hands for each player
    combined_hands = {}
    
    # For each player, combine their hole cards with the table cards
    for player, hand in player_hands.items():
        # Start with the player's hole cards
        combined_hands[player] = hand.copy()
        # Add the table cards
        for card in table_cards.values():
            combined_hands[player].append(card)
            
    return combined_hands

def add_ranking_bracket(combined_hands):
    # Add empty ranking bracket for each player's hand
    ranked_hands = {}
    for player, hand in combined_hands.items():
        ranked_hands[player] = [hand, []]
    return ranked_hands

def pair_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks
        
        # Count occurrences of each rank
        rank_counts = {}
        for rank in ranks:
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:
                rank_counts[rank] = 1
        
        # Find pairs (rank appears exactly twice)
        pairs = [rank for rank, count in rank_counts.items() if count == 2]
        
        # Update ranking based on number of pairs found
        if len(pairs) == 0:
            # No pairs found - leave empty bracket
            continue
        elif len(pairs) == 1:
            # One pair found - add "One Pair" ranking (rank 2)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 2, 'Hand'].values[0]
        elif len(pairs) >= 2:
            # Two or more pairs found - add "Two Pair" ranking (rank 3)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 3, 'Hand'].values[0]
    
    return ranked_hands

def tripple_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks    
        
        # Count occurrences of each rank
        rank_counts = {}
        for rank in ranks:
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:
                rank_counts[rank] = 1
        
        # Find trips (rank appears exactly three times)
        trips = [rank for rank, count in rank_counts.items() if count == 3]
        
        # Update ranking based on number of trips found
        if len(trips) == 0:
            # No trips found - leave empty bracket
            continue
        elif len(trips) == 1:
            # One trip found - add "Three of a Kind" ranking (rank 4)   
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 4, 'Hand'].values[0]
        elif len(trips) >= 2:
            # Two or more trips found - add "Full House" ranking (rank 7)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 7, 'Hand'].values[0]
    
    return ranked_hands

def straight_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks
        
        # Sort ranks in ascending order
        ranks.sort()
        
        # Check for straight
        is_straight = False
        for i in range(len(ranks) - 4): 
            if all(ranks[i+j] == ranks[i] + j for j in range(5)):
                is_straight = True
                break
        
        # Update ranking based on straight status
        if is_straight: 
            # Add "Straight" ranking (rank 5)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 5, 'Hand'].values[0]
    
    return ranked_hands 

def flush_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks
        suits = [card[0] for card in hand]  # Extract just the suits
        
        # Count occurrences of each suit
        suit_counts = {}
        for suit in suits:                  
            if suit in suit_counts:
                suit_counts[suit] += 1
            else:
                suit_counts[suit] = 1
        
        # Check for flush
        is_flush = False
        for suit, count in suit_counts.items():
            if count >= 5:
                is_flush = True
                break   
        
        # Update ranking based on flush status
        if is_flush:
            # Add "Flush" ranking (rank 6)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 6, 'Hand'].values[0]
    
    return ranked_hands 

def full_house_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks        

        # Count occurrences of each rank
        rank_counts = {}
        for rank in ranks:
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:           
                rank_counts[rank] = 1
        
        # Check for full house
        is_full_house = False
        if len(rank_counts) == 2:
            # Full house requires two different ranks   
            trips = [rank for rank, count in rank_counts.items() if count == 3]
            pairs = [rank for rank, count in rank_counts.items() if count == 2]
            if len(trips) == 1 and len(pairs) == 1:
                is_full_house = True
                break   
        
        # Update ranking based on full house status
        if is_full_house:
            # Add "Full House" ranking (rank 7)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 7, 'Hand'].values[0]
    
    return ranked_hands 

def quads_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks

        # Count occurrences of each rank
        rank_counts = {}
        for rank in ranks:
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:   
                rank_counts[rank] = 1
        
        # Check for quads (rank appears exactly four times)
        quads = [rank for rank, count in rank_counts.items() if count == 4] 
        
        # Update ranking based on quads status
        if len(quads) == 1:
            # One quad found - add "Four of a Kind" ranking (rank 8)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 8, 'Hand'].values[0]
    
    return ranked_hands

def straight_flush_search(ranked_hands, hand_rankings_df):
    # For each player's hand
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks
        suits = [card[0] for card in hand]  # Extract just the suits

        # Check for straight flush
        is_straight_flush = False
        for i in range(len(ranks) - 4):
            if all(ranks[i+j] == ranks[i] + j for j in range(5)):
                # Check for flush
                if len(set(suits)) == 1:    
                    is_straight_flush = True
                    break
        
        # Update ranking based on straight flush status
        if is_straight_flush:
            # Add "Straight Flush" ranking (rank 9) 
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 9, 'Hand'].values[0]
    
    return ranked_hands

def royal_flush_search(ranked_hands, hand_rankings_df):
    # For each player's hand    
    for player, hand_data in ranked_hands.items():
        hand = hand_data[0]  # Get the hand (list of [suit,rank] pairs)
        ranks = [card[1] for card in hand]  # Extract just the ranks
        suits = [card[0] for card in hand]  # Extract just the suits

        # Check for royal flush (A, K, Q, J, 10)
        is_royal_flush = False
        if all(rank in ranks for rank in ['A', 'K', 'Q', 'J', '10']):
            # Check for flush
            if len(set(suits)) == 1:
                is_royal_flush = True
                break   
        
        # Update ranking based on royal flush status
        if is_royal_flush:
            # Add "Royal Flush" ranking (rank 10)
            hand_data[1] = hand_rankings_df.loc[hand_rankings_df['Rank'] == 10, 'Hand'].values[0]
    
    return ranked_hands




In [6]:
# Shuffle the deck
shuffled_df = deck_shuffler(deck_df)
player_hands, remaining_deck = initial_deal(shuffled_df, player_hands)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_flop(remaining_deck, table_cards)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_turn(remaining_deck, table_cards)
remaining_deck = burn_card(remaining_deck)
table_cards, remaining_deck = deal_river(remaining_deck, table_cards)
combined_hands = assemble_hand(player_hands, table_cards)
ranked_hands = add_ranking_bracket(combined_hands)
ranked_hands = pair_search(ranked_hands, hand_rankings_df)
ranked_hands = tripple_search(ranked_hands, hand_rankings_df)
ranked_hands = straight_search(ranked_hands, hand_rankings_df)
ranked_hands = flush_search(ranked_hands, hand_rankings_df)
ranked_hands = full_house_search(ranked_hands, hand_rankings_df)
ranked_hands = quads_search(ranked_hands, hand_rankings_df)
ranked_hands = straight_flush_search(ranked_hands, hand_rankings_df)
ranked_hands = royal_flush_search(ranked_hands, hand_rankings_df)



In [7]:
ranked_hands

{0: [[[4, 13], [1, 11], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]], 'One Pair'],
 1: [[[3, 8], [4, 4], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]], 'One Pair'],
 2: [[[4, 11], [1, 4], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]], 'One Pair'],
 3: [[[1, 8], [2, 5], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]], 'One Pair'],
 4: [[[2, 8], [2, 6], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]], 'Flush']}

In [8]:
combined_hands

{0: [[4, 13], [1, 11], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]],
 1: [[3, 8], [4, 4], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]],
 2: [[4, 11], [1, 4], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]],
 3: [[1, 8], [2, 5], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]],
 4: [[2, 8], [2, 6], [1, 5], [2, 2], [4, 3], [2, 4], [2, 13]]}

In [9]:
player_hands

{0: [[4, 13], [1, 11]],
 1: [[3, 8], [4, 4]],
 2: [[4, 11], [1, 4]],
 3: [[1, 8], [2, 5]],
 4: [[2, 8], [2, 6]]}

In [10]:
table_cards

{0: [1, 5], 1: [2, 2], 2: [4, 3], 3: [2, 4], 4: [2, 13]}

In [11]:
remaining_deck

,suit,rank
0,4,2
1,4,8
2,2,9
3,1,13
4,3,11
5,4,7
6,1,2
7,3,7
8,1,9
9,4,6


In [12]:
player_hands

{0: [[4, 13], [1, 11]],
 1: [[3, 8], [4, 4]],
 2: [[4, 11], [1, 4]],
 3: [[1, 8], [2, 5]],
 4: [[2, 8], [2, 6]]}

In [13]:
remaining_deck

,suit,rank
0,4,2
1,4,8
2,2,9
3,1,13
4,3,11
5,4,7
6,1,2
7,3,7
8,1,9
9,4,6


In [14]:
player_hands

{0: [[4, 13], [1, 11]],
 1: [[3, 8], [4, 4]],
 2: [[4, 11], [1, 4]],
 3: [[1, 8], [2, 5]],
 4: [[2, 8], [2, 6]]}

In [15]:
shuffled_df

,suit,rank
50,4,13
32,3,8
48,4,11
6,1,8
19,2,8
9,1,11
41,4,4
2,1,4
16,2,5
17,2,6
